# Stereo SLAM

This is a major part of this project and will likely take some time. 

For stereo, feel free to look up existing tutorials that implement this and write your own code here. Do not spend too long tweaking parameters here, focus on getting decent results and move on. You can also use OpenCV functions to backproject to 3D. 

## PART 1: Stereo dense reconstruction

3-D point clouds are very useful in robotics for several tasks such as object detection, motion estimation (3D-3D matching or 3D-2D matching), SLAM, and other forms of scene understanding.  Stereo camerasprovide  us  with  a  convenient  way  to  generate  dense  point  clouds.Densehere,  in  contrast  tosparse,means all the image points are used for the reconstruction.  In this part of the assignment you will begenerating a dense 3D point cloud reconstruction of a scene from stereo images.

#### Procedure: 

<ol>
    <li> Generate a disparity map for each stereo pair.  Use OpenCV (e.g.  StereoSGBM) for this.  Notethat the images provided are already rectified and undistorted. </li>
    <li> Then, using the camera parameters and baseline information generate colored point clouds from each disparity map.  Some points will have invalid disparity values, so ignore them.  Use [Open3D]for storing your point clouds. </li>
    <li> Register (or transform) all the generated point clouds into your world frame by using the providedground truth poses. </li>
    <li> Visualize the registered point cloud data, in color.  Use Open3D for this </li>
</ol>
    
    
Write briefly about how the disparity map is generated (if you used SGBM, write about SGBM).

In [1]:
import numpy as np
import cv2 as cv
import open3d as o3d
import os

K = np.array([[7.070912e+02, 0.000000e+00, 6.018873e+02],
              [0.000000e+00, 7.070912e+02, 1.831104e+02],
              [0.000000e+00, 0.000000e+00, 1.000000e+00]])
D = 0.53790448812
Ts = np.loadtxt('../data/poses.txt').reshape(-1,3,4)

LIMG_folder = '../data/img2'
RIMG_folder = '../data/img3'
OUT_PATH = '../data/pc'


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [13]:
def get_disparity_map(limg_path, rimg_path, min_disp=16, max_disp=144, win=3):
    limg = cv.cvtColor(cv.imread(limg_path), cv.COLOR_BGR2GRAY)
    rimg = cv.cvtColor(cv.imread(rimg_path), cv.COLOR_BGR2GRAY)

    # https://docs.opencv.org/3.4/d2/d85/classcv_1_1StereoSGBM.html
    disp = cv.StereoSGBM_create(
        minDisparity=min_disp,
        numDisparities=max_disp-min_disp,
        blockSize=7,
        disp12MaxDiff=1,
        speckleWindowSize=400,
        speckleRange=5,
        uniquenessRatio=12,
        P1=8*3*win**2,
        P2=32*3*win**2,
    ).compute(limg, rimg).astype(np.float32) / 16.0

    # to fix -ve disp
    disp += abs(disp.min())
    # from PIL import Image
    # Image.fromarray(disp.astype(np.float32)).show()
    # 1/0
    return disp.astype(np.float32)


def point_to_ply(path, points, colors, disp):
    far_points_mask = disp > disp.min()
    points = points[far_points_mask].reshape(-1, 3)
    colors = colors[far_points_mask].reshape(-1, 3)
    arr = np.c_[points, colors]

    # http://paulbourke.net/dataformats/ply/
    header = (
        "ply\n"
        "format ascii 1.0\n"
        f"element vertex {len(arr)}\n"
        "property float x\n"
        "property float y\n"
        "property float z\n"
        "property uchar red\n"
        "property uchar green\n"
        "property uchar blue\n"
        "end_header\n"
    )
    with open(path, 'w+') as f:
        f.write(header)
        np.savetxt(f, arr, '%f %f %f %d %d %d')

def create_plys():
    os.makedirs(OUT_PATH, exist_ok=True)

    file_names = sorted(os.listdir(LIMG_folder))
    limg_names = [os.path.join(LIMG_folder, file) for file in file_names]
    rimg_names = [os.path.join(RIMG_folder, file) for file in file_names]
    pc_names = [os.path.join(OUT_PATH, file[:-4] + '.ply')
                for file in file_names]
    show = False

    for limg_file, rimg_file, pc_file in zip(limg_names, rimg_names, pc_names):
        disp = get_disparity_map(limg_file, rimg_file)
        limg = cv.imread(limg_file)

        h,w = limg.shape[:2]
        f = K[0][0]
        reverse_P = np.array([
            [1, 0, 0, -0.5*w],
            [0, -1, 0,  0.5*h],
            [0, 0, 0, -f],
            [0, 0, -1/D,  0]
        ], dtype=np.float32)

        coords_3d = cv.reprojectImageTo3D(disp, reverse_P)
        pixel_colors = cv.cvtColor(limg, cv.COLOR_BGR2RGB)
        point_to_ply(pc_file, coords_3d, pixel_colors, disp)

        if show:
            show=False
            from PIL import Image
            Image.fromarray(disp.astype(np.float32)).show()
            
            o3d.visualization.draw_geometries([ 
                o3d.io.read_point_cloud(pc_file)
            ])

        
    
    return pc_names

def combine_pointcloud(pc_names):
    pcd_all = o3d.geometry.PointCloud()
    for T,pc_file in zip(Ts, sorted(pc_names)):
        pcd = o3d.io.read_point_cloud(pc_file)

        # print(hash(np.vstack((T, [0, 0, 0, 1])).tobytes()), pc_file)
        # 1/0
        pcd = pcd.transform(np.vstack((T, [0, 0, 0, 1])))
        pcd_all += pcd
        # o3d.visualization.draw_geometries([pcd])

    
    return pcd_all


In [12]:
ply_files = create_plys()
pcd = combine_pointcloud(ply_files)

# o3d.visualization.draw_geometries([pcd])


6485632299028077324 ../data/pc/0000000460.ply
-3057979224641129272 ../data/pc/0000000461.ply
1217659333877928692 ../data/pc/0000000462.ply
4970122843109033744 ../data/pc/0000000463.ply
4902059576364824391 ../data/pc/0000000464.ply
-1606360050701038974 ../data/pc/0000000465.ply
-5068851243062769037 ../data/pc/0000000466.ply
-5179365858755805749 ../data/pc/0000000467.ply
7639316654742830801 ../data/pc/0000000468.ply
-2497898988907370612 ../data/pc/0000000469.ply
-113287478862950688 ../data/pc/0000000470.ply
4474418534762525133 ../data/pc/0000000471.ply
-6943899959433843772 ../data/pc/0000000472.ply
-6267120774025562933 ../data/pc/0000000473.ply
883472101071728597 ../data/pc/0000000474.ply
-1433336757905145234 ../data/pc/0000000475.ply
4755536964372518397 ../data/pc/0000000476.ply
5456289217364108868 ../data/pc/0000000477.ply
3756135856296457587 ../data/pc/0000000478.ply
-5059046313368155191 ../data/pc/0000000479.ply
-890706848370055466 ../data/pc/0000000480.ply


## PART 2: Motion estimation using iterative PnP

Using the generated reconstruction from the previous part, synthesize a new image taken by a virtualmonocular camera fixed at any arbitrary position and orientation.  Your task in this part is to recoverthis pose using an iterative Perspective-from-n-Points (PnP) algorithm. 

#### Procedure: 

<ol>
    <li> Obtain a set of 2D-3D correspondences between the the image and the point cloud.  Since hereyou’re generating the image, this should be easy to obtain. </li>
    <li> For this set of correspondences compute the total reprojection error c= $\sum_{i} ‖x_i−P_{k}X_i‖^2 $    where $P_{k}= K[R_{k}|t_{k}]$, $X_{i}$ is the 3D point in the world frame, $x_{i}$ is its corresponding projection. </li>
    <li> Solve for the pose $T_{k}$ that minimizes this non-linear reprojection error using a Gauss-Newton (GN)scheme.  Recall that in GN we start with some initial estimated value $x_{o}$ and iteratively refine the estimate using $x_{1}$= $∆x+x_0$, where $∆x$ is obtained by solving the normal equations $J^{T}J∆x$= -$J^{T}e$, until convergence.The main steps in this scheme are computing the corresponding Jacobians and updating the estimates correctly.  For our problem,  use a 12×1 vector parameterization for $T_{k}$(the top 3×4submatrix).  Run the optimization for different choices of initialization and report your observations. </li>
</ol>

Make sure that you write about how you calculate the residual and jacobians. Do not just include the code. The pose that you have been given is the ground truth, so using that will obviously give good results for optimization, so try out something else as well.

## PART 3: Odometry Calculation

In part 1, you used the ground truth pose for registration. Here, try to estimate the pose using the RGB image data alone. 

#### Procedure:

1. This can be done by computing features across the two images and matching them. Since you already have the depth map, you now have correspondences between the depth maps of two images as well from the RGB feature matches. 
2. You can now convert this depth map to a point cloud.
3. Since you have correspondences between image points in the depth map, you have 3D correspondences here as well. Perform ICP here to get a good pose estimate.
4. Feed these initial pose estimates into the PnP pipeline and optimise further.